### Bloco 1: Importação de bibliotecas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time

print("Bibliotecas importadas com sucesso!")

### Bloco 2: Inicialização do WebDriver

In [ ]:
options = Options()
options.add_argument("--start-maximized")  # Abre o navegador maximizado
service = Service(ChromeDriverManager().install())  # Instala o driver automaticamente
driver = webdriver.Chrome(service=service, options=options)
print("WebDriver inicializado com sucesso!")

### Bloco 3: Função de login

In [ ]:
import os

def realizar_login(driver, email, password):
    """
    Realiza o login no Hostinger Mail.
    """
    try:
        print("Abrindo a página de login...")
        driver.get("https://mail.hostinger.com")

        email_field = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.NAME, "_user"))
        )
        email_field.send_keys(email)

        password_field = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.NAME, "_pass"))
        )
        password_field.send_keys(password)

        login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
        )
        login_button.click()

        WebDriverWait(driver, 20).until(
            EC.url_changes("https://mail.hostinger.com")
        )
        print("Login realizado com sucesso!")
    except Exception as e:
        print(f"Erro durante o login: {e}")

# Pegue as credenciais do ambiente
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

realizar_login(driver, email, password)

### Bloco 4: Função para clicar na pasta

In [ ]:
def clicar_na_pasta(driver, nome_pasta):
    """
    Clica na pasta especificada pelo título ou texto visível.
    """
    try:
        print(f"Procurando pela pasta com o título: {nome_pasta} ...")

        # XPath atualizado para localizar pelo título ou texto visível
        xpath = f"//a[@title='{nome_pasta}' or text()='{nome_pasta}']"

        # Aguarda o carregamento da página e a presença do elemento
        pasta_elemento = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )

        # Aguarda até que o elemento seja clicável
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))

        # Role até o elemento e clique
        driver.execute_script("arguments[0].scrollIntoView(true);", pasta_elemento)
        driver.execute_script("arguments[0].click();", pasta_elemento)

        # Aguarda o carregamento da nova página/pasta
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//tr[contains(@class,'message')]"))
        )

        print(f"Pasta com o título '{nome_pasta}' acessada com sucesso.")
    except Exception as e:
        print(f"Erro ao acessar a pasta com o título '{nome_pasta}': {e}")

clicar_na_pasta(driver, "00147 - ADVOCACIA MURILLO DE ARAGÃO")

### Bloco 5: Processar emails do mês atual com anexos

In [ ]:

try:
    tr_element = WebDriverWait(driver,20).until(
        EC.element_to_be_clickable((By.XPATH,"//tr[contains(@class, 'message')]"))
    )

    driver.execute_script("arguments[0].scrollIntoView(true);",tr_element)

    tr_element.click()
    print("Elemento clicado com sucesso!")
except Exception as e:
    print(f"Erro ao clicar no elemento: {e}")

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def clicar_email_e_baixar_anexo(driver):
    """
    Localiza o email desejado, clica no link que contém a palavra 'ADVOCACIA',
    e depois clica no anexo dentro do email aberto.
    """
    try:
        print("Procurando pelo email com as condições especificadas...")

        # Localiza todas as linhas de email
        emails = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, "//tr[contains(@class, 'message')]"))
        )
        
        for email in emails:
            # Verifica se a linha do email possui a classe 'message selected focused'
            if "selected focused" in email.get_attribute("class"):
                # Verifica se há um 'td' com classe 'subject'
                td_subject = email.find_element(By.CLASS_NAME, "subject")
                # Verifica se há um 'span' com classe 'subject' dentro do 'td'
                span_subject = td_subject.find_element(By.CLASS_NAME, "subject")
                # Verifica se o 'a' dentro do 'span' contém a palavra 'ADVOCACIA'
                link = span_subject.find_element(By.TAG_NAME, "a")
                href = link.get_attribute("href")
                if "ADVOCACIA" in href:
                    # Role para o elemento
                    driver.execute_script("arguments[0].scrollIntoView(true);", link)
                    
                    # Pausa para garantir a renderização
                    WebDriverWait(driver, 10).until(EC.visibility_of(link))
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(link))
                    
                    # Clica no link para abrir o email
                    driver.execute_script("arguments[0].click();", link)
                    print(f"Link com a palavra 'ADVOCACIA' encontrado e clicado: {href}")
                    
                    # Aguarda o carregamento da página do email
                    WebDriverWait(driver, 30).until(
                        EC.presence_of_element_located((By.XPATH, "//span[@class='attachment-name']"))
                    )
                    
                    # Localiza e clica no anexo
                    anexo = WebDriverWait(driver, 30).until(
                        EC.element_to_be_clickable((By.XPATH, "//span[@class='attachment-name']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", anexo)
                    driver.execute_script("arguments[0].click();", anexo)
                    
                    print("Anexo encontrado e clicado com sucesso!")
                    return  # Encerra após concluir a tarefa
                
        print("Nenhum email com as condições especificadas foi encontrado.")
    except Exception as e:
        print(f"Erro durante o processo: {e}")

# Chame a função
clicar_email_e_baixar_anexo(driver)
